In [3]:
from datetime import datetime 
start_real = datetime.now()
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
# from keras.layers import Bidirectional
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from nltk.corpus import stopwords
import math
# set seed
np.random.seed(123)

Using TensorFlow backend.


In [4]:
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y )))

def root_mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.sqrt(K.mean(K.square(first_log - second_log), axis=-1)+0.0000001)

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)+0.0000001)


In [5]:
%%time

FLAG_LOAD_TEST = False
FLAG_MAKE_SAMPLE = True

train_df = pd.read_table('input/train.tsv')
if FLAG_MAKE_SAMPLE:
    train_df = train_df.sample(10000)
print(train_df.shape)

if FLAG_LOAD_TEST:
    test_df = pd.read_table('input/test.tsv')
    print(test_df.shape)

(10000, 8)


NameError: name 'test_df' is not defined

In [6]:
# remove low prices
train_df = train_df.drop(train_df[(train_df.price < 3.0)].index)
train_df.shape

(9995, 8)

In [7]:
%%time
# get name and description lengths
def wordCount(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except: 
        return 0
train_df['desc_len'] = train_df['item_description'].apply(lambda x: wordCount(x))
train_df['name_len'] = train_df['name'].apply(lambda x: wordCount(x))

if FLAG_LOAD_TEST:
    test_df['desc_len'] = test_df['item_description'].apply(lambda x: wordCount(x))
    test_df['name_len'] = test_df['name'].apply(lambda x: wordCount(x))

train_df.head()

Wall time: 45 ms


In [8]:
%%time
# split category name into 3 parts
def split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")
train_df['subcat_0'], train_df['subcat_1'], train_df['subcat_2'] = \
zip(*train_df['category_name'].apply(lambda x: split_cat(x)))

if FLAG_LOAD_TEST:
    test_df['subcat_0'], test_df['subcat_1'], test_df['subcat_2'] = \
    zip(*test_df['category_name'].apply(lambda x: split_cat(x)))

Wall time: 18 ms


In [9]:
%%time


full_set = pd.concat([train_df])
if FLAG_LOAD_TEST:
    full_set = pd.concat([train_df,test_df])
    
all_brands = set(full_set['brand_name'].values)
train_df.brand_name.fillna(value="missing", inplace=True)

if FLAG_LOAD_TEST:
    test_df.brand_name.fillna(value="missing", inplace=True)

# get to finding!
premissing = len(train_df.loc[train_df['brand_name'] == 'missing'])
def brandfinder(line):
    brand = line[0]
    name = line[1]
    namesplit = name.split(' ')
    if brand == 'missing':
        for x in namesplit:
            if x in all_brands:
                return name
    if name in all_brands:
        return name
    return brand
train_df['brand_name'] = train_df[['brand_name','name']].apply(brandfinder, axis = 1)

if FLAG_LOAD_TEST:
    test_df['brand_name'] = test_df[['brand_name','name']].apply(brandfinder, axis = 1)
found = premissing-len(train_df.loc[train_df['brand_name'] == 'missing'])
print(found)

420
Wall time: 226 ms


Standard split the train test for validation and log the price

In [10]:
# Scale target variable to log.
train_df["target"] = np.log1p(train_df.price)

# Split training examples into train/dev examples.
train_df, dev_df = train_test_split(train_df, test_size=0.2, random_state =102)

# Calculate number of train/dev/test examples.
n_trains = train_df.shape[0]
n_devs = dev_df.shape[0]

print("Training on", n_trains, "examples")
print("Validating on", n_devs, "examples")

if FLAG_LOAD_TEST:
    n_tests = test_df.shape[0]
    print("Testing on", n_tests, "examples")

Training on 7996 examples
Validating on 1999 examples


# Ridge Models

Now onto the Ridge models. Less to play with in the Ridge models but it is faster than the RNN. 

In [11]:
# Concatenate train - dev - test data for easy to handle

if FLAG_LOAD_TEST:
    full_df = pd.concat([train_df, dev_df, test_df])
else:
    full_df = pd.concat([train_df, dev_df])


## Handle missing data and convert data type to string
All inputs must be strings in a ridge model. The other note here is that filling NAs for item_description use 'No description yet' so it is read the same as the 'No description yet' entries.

In [12]:
%%time

print("Handling missing values...")
full_df['category_name'] = full_df['category_name'].fillna('missing').astype(str)
full_df['subcat_0'] = full_df['subcat_0'].astype(str)
full_df['subcat_1'] = full_df['subcat_1'].astype(str)
full_df['subcat_2'] = full_df['subcat_2'].astype(str)
full_df['brand_name'] = full_df['brand_name'].fillna('missing').astype(str)
full_df['shipping'] = full_df['shipping'].astype(str)
full_df['item_condition_id'] = full_df['item_condition_id'].astype(str)
full_df['desc_len'] = full_df['desc_len'].astype(str)
full_df['name_len'] = full_df['name_len'].astype(str)
full_df['item_description'] = full_df['item_description'].fillna('No description yet').astype(str)

Handling missing values...
Wall time: 89.5 ms


## Vectorizing all the data
Takes around 8-10 minutes depending on the inputs used.

In [13]:
%%time

print("Vectorizing data...")
default_preprocessor = CountVectorizer().build_preprocessor()
def build_preprocessor(field):
    field_idx = list(full_df.columns).index(field)
    return lambda x: default_preprocessor(x[field_idx])

vectorizer = FeatureUnion([
    ('name', CountVectorizer(
        ngram_range=(1, 2),
        max_features=50000,
        preprocessor=build_preprocessor('name'))),
#     ('category_name', CountVectorizer(
#         token_pattern='.+',
#         preprocessor=build_preprocessor('category_name'))),
    ('subcat_0', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_0'))),
    ('subcat_1', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_1'))),
    ('subcat_2', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('subcat_2'))),
    ('brand_name', CountVectorizer(
        token_pattern='.+',
        preprocessor=build_preprocessor('brand_name'))),
    ('shipping', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('shipping'))),
    ('item_condition_id', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('item_condition_id'))),
    ('desc_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('desc_len'))),
    ('name_len', CountVectorizer(
        token_pattern='\d+',
        preprocessor=build_preprocessor('name_len'))),
    ('item_description', TfidfVectorizer(
        ngram_range=(1, 3),
        max_features=100000,
        preprocessor=build_preprocessor('item_description'))),
])

X = vectorizer.fit_transform(full_df.values)

X_train = X[:n_trains]
Y_train = train_df.target.values.reshape(-1, 1)

X_dev = X[n_trains:n_trains+n_devs]
Y_dev = dev_df.target.values.reshape(-1, 1)

X_test = X[n_trains+n_devs:]
print(X.shape, X_train.shape, X_dev.shape, X_test.shape)

Vectorizing data...
(9995, 132730) (7996, 132730) (1999, 132730) (0, 132730)
Wall time: 1.95 s


## Fitting Ridge model on training data
A Ridge model with cross validation does *slighly* better than one without, but even with the minimum of 2 CV, it still takes 4-5 minutes. Any more and it becomes impractical for our narrow time limit. A regular ridge model will only take ~30 seconds. So, for the purposes of having another model to predict on, might as well make both.

In [14]:
%%time

print("Fitting Ridge model on training examples...")
ridge_model = Ridge(
    solver='auto', fit_intercept=True, alpha=1.0,
    max_iter=100, normalize=False, tol=0.05, random_state = 1,
)
ridge_modelCV = RidgeCV(
    fit_intercept=True, alphas=[5.0],
    normalize=False, cv = 2, scoring='neg_mean_squared_error',
)
ridge_model.fit(X_train, Y_train)
ridge_modelCV.fit(X_train, Y_train)

Fitting Ridge model on training examples...
Wall time: 914 ms


## Evaluating Ridge model on dev data

In [15]:
Y_dev_preds_ridge = ridge_model.predict(X_dev)
Y_dev_preds_ridge = Y_dev_preds_ridge.reshape(-1, 1)
print("RMSL error on dev set:", rmsle(Y_dev, Y_dev_preds_ridge))

RMSL error on dev set: 0.547373677969


In [16]:
Y_dev_preds_ridgeCV = ridge_modelCV.predict(X_dev)
Y_dev_preds_ridgeCV = Y_dev_preds_ridgeCV.reshape(-1, 1)
print("CV RMSL error on dev set:", rmsle(Y_dev, Y_dev_preds_ridgeCV))

CV RMSL error on dev set: 0.54132434514


## Predicting results

In [44]:
y_pred = ridge_model.predict(X_test)

y_pred[y_pred<1.386]=1.386
y_pred = np.expm1(y_pred)

sub = pd.DataFrame()
sub['test_id'] =test_df.test_id.astype(int)
sub['price'] = y_pred
sub.to_csv('result.csv', index=False)
print ("fini")

fini


# References

This was originally based off of this Kernal: https://www.kaggle.com/nvhbk16k53/associated-model-rnn-ridge

With ideas gained from the visualizations here: https://www.kaggle.com/thykhuely/mercari-interactive-eda-topic-modelling

You can find description of the competition here https://www.kaggle.com/c/mercari-price-suggestion-challenge